# training:valid=9:1:1장 테스트 결과

In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion()   # 대화형 모드

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print("------- GPU Working -------")
    print("[Current GPU]: " + str(torch.cuda.get_device_name(0)))
else:
    print("------- CPU Working -------")
print('[PyTorch version]: ', torch.__version__)

------- GPU Working -------
[Current GPU]: NVIDIA GeForce RTX 3090
[PyTorch version]:  1.11.0


In [3]:
batch_size=1
num_workers=8  #16안좋은듯

In [4]:
folder_path = 'D:\\experiment_dataset'
folder_list = os.listdir(folder_path)
subject_list=folder_list

In [15]:
subject_list

['F1',
 'F10',
 'F11',
 'F12',
 'F13',
 'F14',
 'F15',
 'F16',
 'F17',
 'F18',
 'F19',
 'F2',
 'F20',
 'F21',
 'F22',
 'F23',
 'F24',
 'F25',
 'F26',
 'F27',
 'F28',
 'F29',
 'F3',
 'F30',
 'F31',
 'F32',
 'F33',
 'F34',
 'F35',
 'F36',
 'F37',
 'F4',
 'F5',
 'F6',
 'F7',
 'F8',
 'F9',
 'M1',
 'M10',
 'M11',
 'M12',
 'M13',
 'M14',
 'M15',
 'M16',
 'M17',
 'M18',
 'M19',
 'M2',
 'M20',
 'M21',
 'M22',
 'M23',
 'M24',
 'M25',
 'M26',
 'M27',
 'M28',
 'M29',
 'M3',
 'M30',
 'M31',
 'M32',
 'M33',
 'M34',
 'M35',
 'M36',
 'M37',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9']

In [16]:
subject_list.index('F30')

23

In [18]:
test_transform = torchvision.transforms.Compose([
                                transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

test_loader_set = []

for one_subject in subject_list:
    # try:
    test_path= 'D:\\experiment_dataset\\'  +  '{}\\test'.format(one_subject)
    test_dataset = torchvision.datasets.ImageFolder(test_path, transform=test_transform)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=num_workers)
    test_loader_set.append(test_loader)
    # except:
    #     pass
    # break
    class_names = 7
    learning_rate =0.1 #1e-5(X)

    model=torchvision.models.resnet152(pretrained=True)

    num_ftrs = model.fc.in_features 
    model.fc = torch.nn.Linear(num_ftrs, class_names) #(fc): Linear(in_features=2048, out_features=7, bias=True)
    model.to(device)

    model_save_dir= 'D:\\Task/Resnet152/saved_model\\' + '{}.pt'.format(one_subject)

    optimizer = torch.optim.SGD(model.fc.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)

    checkpoint = torch.load(model_save_dir)

    model.load_state_dict(checkpoint['model_state_dict'])#checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']

    model.eval()
    
    total_loss = 0
    total_len = 0
    total_correct = 0

    for batch_sample, batch_label in test_loader:

        sample = batch_sample.to(device)
        label = batch_label.to(device)

        with torch.no_grad():
            output = model.forward(sample) 
        # loss = loss_func(output, label)     

        _, output_index = torch.max(output, 1)   
        preds = output_index
       
        correct = preds.eq(label)   #1배치에 맞힌 갯수     

        total_correct += correct.sum().item()  #1배치에 맞힌 개수 계속 더함        

        total_len += len(label)      #1배치 갯수 계속 더함

        test_accuracy = total_correct / total_len #이시점까지(에폴당) 누적된(배치마다 계속 맞힌 갯수 더한) 정확도
        # print('test_accuracy', test_accuracy)
    # print('{}/{}  Test accuracy: {:.4f}'.format(epoch + 1, epochs, test_accuracy)) #(에폭당) 전체데이터셋에 대한 정확도 #마지막 배치(앞에서부터 끝까지 누적됨)정확도랑 같음
    print('{} Test accuracy: {:.4f}'.format(one_subject, test_accuracy)) 

F1 Test accuracy: 0.7143
F10 Test accuracy: 0.8571
F11 Test accuracy: 1.0000
F12 Test accuracy: 0.8571
F13 Test accuracy: 0.8571
F14 Test accuracy: 0.7143
F15 Test accuracy: 0.7143
F16 Test accuracy: 1.0000
F17 Test accuracy: 0.0000
F18 Test accuracy: 0.8571
F19 Test accuracy: 1.0000
F2 Test accuracy: 0.8571
F20 Test accuracy: 1.0000
F21 Test accuracy: 1.0000
F22 Test accuracy: 1.0000
F23 Test accuracy: 0.8571
F24 Test accuracy: 0.7143
F25 Test accuracy: 0.8571
F26 Test accuracy: 1.0000
F27 Test accuracy: 1.0000
F28 Test accuracy: 0.7143
F29 Test accuracy: 1.0000
F3 Test accuracy: 1.0000
F30 Test accuracy: 0.8571
F31 Test accuracy: 0.8571
F32 Test accuracy: 1.0000
F33 Test accuracy: 0.7143
F34 Test accuracy: 1.0000
F35 Test accuracy: 1.0000
F36 Test accuracy: 1.0000
F37 Test accuracy: 1.0000
F4 Test accuracy: 0.2857
F5 Test accuracy: 1.0000
F6 Test accuracy: 0.2857
F7 Test accuracy: 1.0000
F8 Test accuracy: 1.0000
F9 Test accuracy: 1.0000
M1 Test accuracy: 0.8571
M10 Test accuracy: 0.8

In [ ]:
F1 Test accuracy: 0.7143
F10 Test accuracy: 0.8571
F11 Test accuracy: 1.0000
F12 Test accuracy: 0.8571
F13 Test accuracy: 0.8571
F14 Test accuracy: 0.7143
F15 Test accuracy: 0.7143
F16 Test accuracy: 1.0000
F17 Test accuracy: 0.0000
F18 Test accuracy: 0.8571
F19 Test accuracy: 1.0000
F2 Test accuracy: 0.8571
F20 Test accuracy: 1.0000
F21 Test accuracy: 1.0000
F22 Test accuracy: 1.0000
F23 Test accuracy: 0.8571
F24 Test accuracy: 0.7143
F25 Test accuracy: 0.8571
F26 Test accuracy: 1.0000
F27 Test accuracy: 1.0000
F28 Test accuracy: 0.7143
F29 Test accuracy: 0.8571